# ChipIR Fluence calculation

We start by importing all of the required libraries and external functions.

Then we select the appropriate input files.

In [10]:
from datetime import datetime, date, time, timedelta
import os
import re
import pandas as pd
import numpy as np

from calculate_flux import get_fluency_flux
from calculate_flux import read_count_file
from calculate_flux import calculate_open_periods

# Directory path
directory = "./"

# Distance file retrieved from measurements at the location
distance_factor_file = "distance_file.txt"

# Devices of interest
allowed_devices = ['bruno1', 'bruno2']

# File containing the shutter open/close information
shutter_file = "shutter.txt"

# Time window for each run
time_unit_offset = timedelta(hours=0.1)

## Filtering log data

Filter all of the ```countlog``` files. These are retrieved from the facility directly. 

This should filter malformed or broken lines.

In [11]:
# Get all countlog files in the directory
countlog_files = [filename for filename in os.listdir(directory) if filename.startswith("countlog")]

# Sort the countlog files in chronological order
countlog_files.sort(key=lambda x: datetime.strptime(re.search(r"\d{4}-\d{2}-\d{2}", x).group(), "%Y-%m-%d"))

# Initialize an empty dataframe
neutron_count_df = pd.DataFrame()

# Loop through the countlog files in chronological order
for filename in countlog_files:
    # Get the full file path
    file_path = os.path.join(directory, filename)

    print(filename)
    # Read the count file and append it to the dataframe
    neutron_count = read_count_file(file_path)
    neutron_count_df = pd.concat([neutron_count_df, pd.DataFrame(neutron_count)]).reset_index(drop=True)

neutron_count_df = neutron_count_df.rename(columns={0: "Timestamp", 1: "Fission Count"})

# Load all distances before continue
distance_data = pd.read_csv(distance_factor_file, sep="\t")

countlog-2023-12-04.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-05.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-06.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-07.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-08.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-09.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-10.txt
Ignoring line (malformed):Data From ChipIR



Now we can remove the logs relative to times where the boards were not in the table (distance file)

In [12]:
#Filter out the data that is not in the distance file for each device
devices_dataframe = {}

for device in distance_data["board"].unique():
    if device in allowed_devices:
        distance_line = distance_data[distance_data["board"] == device]
        start_line = distance_line["start"].iloc[0]
        end_line = distance_line["end"].iloc[0]

        start_dt = pd.to_datetime(start_line, format='%d/%m/%Y %H:%M:%S')
        end_dt = pd.to_datetime(end_line, format='%d/%m/%Y %H:%M:%S')
        
        filtered_df = neutron_count_df[
            (neutron_count_df["Timestamp"] >= start_dt) &
            (neutron_count_df["Timestamp"] <= end_dt)
        ]
        
        devices_dataframe[device] = filtered_df

## Calculate the fluence

Calculate the fluence for each device in ```allowed_devices``` that we selected at the beggining.

Store it in ```device_dict``` dictionary, the keys are ```allowed_devices```. The fields for each device are as follows:

| Start Date | Flux | Time Beam Off | Fluence |
|------------|------|---------------|---------|
| 2023-12-05 21:00:00.403 | 0.0 | 353.895         | 0.0     |

In [13]:

device_dict = {}

for device in allowed_devices:
    device_flunce_pd = pd.DataFrame(columns=['Start Date', 'Flux', 'Time Beam Off', 'Fluence'])
    
    df = devices_dataframe[device]

    distance_line = distance_data[
        (distance_data["board"].str.contains(device))
    ]
    facility_factor = float(distance_line["facility_factor"])
    distance_attenuation = float(distance_line["Distance attenuation"])

    start_dt = df["Timestamp"].iloc[0]

    while start_dt <= df["Timestamp"].iloc[-1]:
        end_dt = start_dt + time_unit_offset
        
        if end_dt > df["Timestamp"].iloc[-1]:
            break

        mistake_time = pd.to_datetime("2023-12-08 10:23:00")
        if start_dt <= mistake_time and end_dt >= mistake_time:
            # print("aaaaaa",start_dt, end_dt,mistake_time)
            start_dt = start_dt + time_unit_offset
            continue

        neutron_count = df[(df["Timestamp"] >= start_dt) & (df["Timestamp"] <= end_dt)].values
        if len(neutron_count) == 0:
            start_dt = start_dt + time_unit_offset
            continue
        

        flux, time_beam_off = get_fluency_flux(
            start_dt=start_dt,
            end_dt=end_dt,
            neutron_count=neutron_count,
            facility_factor=facility_factor,
            distance_attenuation=distance_attenuation,
        )
        
        fluence = flux * time_unit_offset.total_seconds()  
        data = [
            [start_dt, flux, time_beam_off, flux * time_unit_offset.total_seconds()]
        ]
        device_flunce_pd = pd.concat([device_flunce_pd, pd.DataFrame(data, columns=['Start Date', 'Flux', 'Time Beam Off', 'Fluence'])], ignore_index=True)
        # print(start_dt, flux, time_beam_off, fluence)

        start_dt = end_dt
    
    device_dict[device] = device_flunce_pd

 



## Display Fluence data

The data in ```device_dict``` contains the fluence. Now we display overall statistics of beam and all devices.

In [14]:
def display_periods(periods):
    """ Display the open periods """
    total_duration = timedelta()
    for start, end in periods:
        total_duration += end - start
        
    total_duration_hours = total_duration.total_seconds() / 3600
    print(f"Shutter open for a total time of {total_duration_hours} hours")

def largest_stretch(df):
    # Find the largest continuous stretch of rows without discontinuity
    max_stretch = 0
    current_stretch = 0
    start_index = 0
    end_index = 0

    for i in range(len(df) - 1):
        if df.index[i] + 1 == df.index[i + 1]:
            current_stretch += 1
        else:
            if current_stretch > max_stretch:
                max_stretch = current_stretch
                start_index = i - current_stretch
                end_index = i
            current_stretch = 0

    if current_stretch > max_stretch:
        max_stretch = current_stretch
        start_index = len(filtered_df) - current_stretch
        end_index = len(filtered_df) - 1

    return df.iloc[start_index:end_index + 1]

periods = calculate_open_periods(shutter_file)
display_periods(periods)

for device in device_dict:
    print(f"\n------------------------- {device} -------------------------")
    device_dict[device] = device_dict[device][device_dict[device]["Flux"] != 0]
    total_time_beam_off = device_dict[device]["Time Beam Off"].sum()
    num_time_windows = device_dict[device].shape[0] * time_unit_offset.total_seconds() / 3600
    total_fluence = device_dict[device]["Fluence"].sum()
    print(f"\t{round(total_time_beam_off/3600, 2)} hours of beam off out of {num_time_windows} hours") # in hours
    print(f"\tTotal Fluence = {total_fluence:.2e}")


Shutter open for a total time of 64.46666666666667 hours

------------------------- bruno1 -------------------------
	3.72 hours of beam off out of 54.3 hours
	Total Fluence = 8.99e+11

------------------------- bruno2 -------------------------
	3.72 hours of beam off out of 54.3 hours
	Total Fluence = 8.85e+11


## Process data

Filter the data frame of each device and filter by start time

In [18]:
# filtered_df = device_dict[allowed_devices[0]][device_dict[allowed_devices[0]]['Time Beam Off'] < 100]
filtered_df = device_dict[allowed_devices[0]]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 7]
filtered_df = filtered_df[filtered_df['Start Date'].dt.hour >=10]
filtered_df = filtered_df[filtered_df['Start Date'].dt.hour < 23]
# filtered_df = filtered_df[filtered_df['Start Date'].dt.minute >= 0]
# filtered_df = filtered_df[filtered_df['Start Date'].dt.minute <=58]


## Example configuration filter

This will filter device 0 from ```allowed_devices``` according to the requirements of the experiment.

This part will vary a lot and depends on what the experiment demands, so only a simple example is provided where we filter the fluence by start date and for windows (```time_unit_offset```) with less than 100 seconds of beam off time.

Then the largest continuous stretch of data is selected. We use it to calculate the mean fluence in that period.

In [28]:
filtered_df = device_dict[allowed_devices[0]][device_dict[allowed_devices[0]]['Time Beam Off'] < 100]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 7]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time > time(10,35)]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time < time(19,0)]
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
display(filtered_df)

print("************** Total for period **************")
print(f"Window : {filtered_df['Start Date'].iloc[0]} -- {filtered_df['Start Date'].iloc[-1]}")
mean_fluence = filtered_df['Fluence'].sum()
print(f"Mean Fluence: {mean_fluence:.2e}")

print("************** Largest Stretch **************")
filter_stretch = largest_stretch(filtered_df)
# largest_stretch
end_date = filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)
print(f"Window : {filter_stretch['Start Date'].iloc[0]} -- {filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)}")
mean_fluence = filter_stretch['Fluence'].sum()
print(f"Mean Fluence: {mean_fluence:.2e}")

,Start Date,Flux,Time Beam Off,Fluence
376,2023-12-07 10:36:00.403,5.056479e+06,0.000,1.820333e+09
377,2023-12-07 10:42:00.403,5.026306e+06,0.000,1.809470e+09
378,2023-12-07 10:48:00.403,5.063496e+06,0.000,1.822859e+09
379,2023-12-07 10:54:00.403,5.009465e+06,0.000,1.803407e+09
380,2023-12-07 11:00:00.403,5.029815e+06,0.000,1.810733e+09
381,2023-12-07 11:06:00.403,4.978590e+06,0.000,1.792292e+09
382,2023-12-07 11:12:00.403,5.041042e+06,0.000,1.814775e+09
383,2023-12-07 11:18:00.403,5.013675e+06,0.000,1.804923e+09
384,2023-12-07 11:24:00.403,5.024903e+06,0.000,1.808965e+09
385,2023-12-07 11:30:00.403,5.049462e+06,0.000,1.817806e+09


************** Total for period **************
Window : 2023-12-07 10:36:00.403000 -- 2023-12-07 18:54:00.403000
Mean Fluence: 1.26e+11
************** Largest Stretch **************
Window : 2023-12-07 10:36:00.403000 -- 2023-12-07 13:06:00.403000
Mean Fluence: 4.52e+10
